In [36]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
import os

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [37]:
cwd = Path.cwd().parent.parent
root_dir = Path( cwd / "01-intro")
def read_data(data_path:str) -> pd.DataFrame:
    data_path = Path(data_path)
    data_dir = root_dir / data_path
    print(data_dir.resolve())
    data01 = pd.read_parquet(data_dir)
    
    return data01

dataset = read_data("assignment/data/yellow_tripdata_2023-01.parquet")
dataset.head()

/Users/alghali/Downloads/AI-Compeitions/mlops-zoomcamp/01-intro/assignment/data/yellow_tripdata_2023-01.parquet


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [38]:
len(dataset.columns.tolist())

19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [39]:
dataset['tpep_pickup_datetime'] = pd.to_datetime(dataset['tpep_pickup_datetime'])
dataset['tpep_dropoff_datetime'] = pd.to_datetime(dataset['tpep_dropoff_datetime'])

dataset['duration'] = (dataset['tpep_dropoff_datetime'] - dataset['tpep_pickup_datetime']).dt.total_seconds() / 60

dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [40]:
dataset['duration'].std()

42.594351241920904

### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%

92%

95%

98%

In [41]:
dataset.shape[0]

3066766

In [42]:
dataset_filtered = dataset[(dataset['duration'] >= 1) & (dataset['duration'] <= 60)]
ratio = dataset_filtered.shape[0] / dataset.shape[0]
ratio

0.9812202822125979


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [43]:
from sklearn.feature_extraction import DictVectorizer

dataset['PULocationID'] = dataset['PULocationID'].astype(str)
dataset['DOLocationID'] = dataset['DOLocationID'].astype(str)

dict = dataset[['PULocationID','DOLocationID']].to_dict(orient='records')
dv = DictVectorizer()
X = dv.fit_transform(dict)

print(f"Dimetionality is", X.shape[1])
len(dict)

Dimetionality is 518


3066766

maybe because i used the sparse=True but in all cases I'm going to answer it as 515

# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64


In [44]:
# did they just made the model determinsitcs
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression 

model = LinearRegression()
y = dataset['duration']
model.fit(X,y)
y_predicted = model.predict(X)
score = root_mean_squared_error(y_predicted,y)
print(score)

41.9964977084757


I'm cooked fr

# Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81


In [45]:
dataset = read_data("assignment/data/yellow_tripdata_2023-02.parquet")



dataset['tpep_pickup_datetime'] = pd.to_datetime(dataset['tpep_pickup_datetime'])
dataset['tpep_dropoff_datetime'] = pd.to_datetime(dataset['tpep_dropoff_datetime'])

dataset['duration'] = (dataset['tpep_dropoff_datetime'] - dataset['tpep_pickup_datetime']).dt.total_seconds() / 60

dataset.head()

from sklearn.feature_extraction import DictVectorizer

dataset['PULocationID'] = dataset['PULocationID'].astype(str)
dataset['DOLocationID'] = dataset['DOLocationID'].astype(str)

dict = dataset[['PULocationID','DOLocationID']].to_dict(orient='records')
dv = DictVectorizer()
X = dv.fit_transform(dict)

print(f"Dimetionality is", X.shape[1])


y_predicted = model.predict(X)
score = root_mean_squared_error(y_predicted,y)
print(score)

/Users/alghali/Downloads/AI-Compeitions/mlops-zoomcamp/01-intro/assignment/data/yellow_tripdata_2023-02.parquet
Dimetionality is 519


ValueError: X has 519 features, but LinearRegression is expecting 518 features as input.

In [ ]:
len(dict)

2913955

# no way there is such a question, I'm gussing it NGL